# **Chargeback FCA add-on**
#### **Fabric Cost Analysis** and **Fabric Unified Admin Monitoring** better together
- Enable chargeback with the **chargeback FCA add-on** to bridge between your Microsoft Fabric Azure costs and capacity CU consumption.
- This allows you to allocate capacity costs by workspace based on actual CU usage!
- Add **Shorcuts and views to the FCA Lakehouse**, ready to use with no refresh required!


#### **To Do**
- Update parameter in cell #2: Name of FUAM and FCA Lakehouse / workspace
- Run the notebook
- The view <u>v_FabricCostSplitByWorkspace</u> will provide a daily breakdown of capacity costs by workspace


### **Beta version**
- Work in progress: add dedicated report page and split by users
- Feedback are welcome!!!

In [ ]:
%pip install semantic-link-labs

In [ ]:
# Parameters for source and destination
source_lakehouse = "FUAM_Lakehouse"
source_workspace = "FUAM"
destination_lakehouse = "FCA"
destination_workspace = "FCA"
schema_shortcut_destination = "dbo"

In [ ]:
# Import Sempy for Shortcut Creation
import json 
import notebookutils 
import sempy.fabric as fabric 
from sempy.fabric.exceptions import FabricHTTPException, WorkspaceNotFoundException 
import sempy_labs.lakehouse as lake
import sempy_labs as labs

In [ ]:
## List of tables for Shortcut creation
# capacities  // capacity_metrics_by_item_kind_by_day // workspaces
tables_shortcut = ["capacities","capacity_metrics_by_item_kind_by_day","workspaces"]
prefix_shortcut = "FUAM_"

In [ ]:
# Function to check for existing shortcuts
def shortcut_exists(destination_lakehouse, destination_workspace, shortcut_name):
    existing_shortcuts = lake.get_lakehouse_tables(
        lakehouse=destination_lakehouse,
        workspace=destination_workspace
    )
    return shortcut_name in existing_shortcuts['Table Name'].values

for table in tables_shortcut:
    shortcut_name = prefix_shortcut + table
    destination_path = destination_path="Tables/" + schema_shortcut_destination
    # Check if the shortcut already exists
    if shortcut_exists(destination_lakehouse, destination_workspace, shortcut_name):
        try:
            # Delete the existing shortcut
            lake.delete_shortcut(
                shortcut_name=shortcut_name,
                lakehouse=destination_lakehouse,
                workspace=destination_workspace
            )
            print(f"🟢 The shortcut '{shortcut_name}' in '{destination_workspace}' has been deleted.")
        except Exception as e:
            print(f"🔴 Failed to delete existing shortcut '{shortcut_name}'. Error: {e}")
            continue  # Skip to the next shortcut if delete fails
        
    # Create a new shortcut
    try:
        lake.create_shortcut_onelake(table_name=table, source_lakehouse=source_lakehouse, 
        source_workspace=source_workspace, destination_lakehouse=destination_lakehouse, 
        destination_workspace=destination_workspace, 
        destination_path=destination_path, shortcut_name=shortcut_name)
        print(f"🟢 Shortcut created successfully for: '{shortcut_name}'")
    except Exception as e:
        print(f"🔴 Failed to create shortcut for '{shortcut_name}'. Error: {e}")

print("All shortcuts processed successfully.")

In [ ]:
#Shorcut verification
df = lake.list_shortcuts(lakehouse=destination_lakehouse, workspace=destination_workspace)
df

In [ ]:
# Create View v_CapacityCostPeriod = Get per day et per capacity: Cost, Capacity ID and workspace #

with labs.ConnectLakehouse(lakehouse=destination_lakehouse, workspace=destination_workspace) as sql:
    query = """
    CREATE OR ALTER View [dbo].[v_CapacityCostPeriod] as 
    SELECT FUAM_capacities.CapacityId,ResourceName as CapacityName,BillingPeriodStart, ChargePeriodStart as ChargePeriod, SUM(BilledCost) as TotalCost, NbWorkspaces
    FROM [dbo].[focus_fabric] cost
    JOIN [dbo].[resources] on cost.ResourceKey = resources.ResourceKey
    JOIN [dbo].[FUAM_capacities] on resources.ResourceName = FUAM_capacities.displayName
    JOIN (SELECT CapacityId,APPROX_COUNT_DISTINCT(WorkspaceId) as NbWorkspaces FROM [dbo].[FUAM_workspaces] GROUP BY CapacityId) as NbWorkspace on FUAM_capacities.CapacityId = NbWorkspace.CapacityId
    GROUP BY FUAM_capacities.CapacityId,ResourceName,BillingPeriodStart, ChargePeriodStart,NbWorkspaces
    """
    # Execute 
    sql.query(query)

In [ ]:
# Create View v_WorkspacesCUConsumption = Get all CU consumption and percentage per Capacity split by workspace 
# If no consumption return null
# Work only for F SKU

with labs.ConnectLakehouse(lakehouse=destination_lakehouse, workspace=destination_workspace) as sql:
    query = """
    CREATE OR ALTER VIEW [dbo].[v_WorkspacesCUConsumption] as 
    SELECT *,
    SUM(SumCUWorkspace) OVER (PARTITION BY CapacityId, CapacityName, DateCU) as TotalWorkspaceCUCapacity,
    -- CALCULATE foreach Capacity, Current Workspace CU Consumption vs SUM of all workspaces CU Consumption
    SumCUWorkspace / SUM(SumCUWorkspace) OVER (PARTITION BY CapacityId, CapacityName, DateCU) as TotalCUWorkspacePercCapacity FROM(
    SELECT FUAM_capacities.CapacityId, FUAM_capacities.displayName as CapacityName, WorkspaceCapacityUsage.WorkspaceId, FUAM_workspaces.WorkspaceName, WorkspaceCapacityUsage.Date as DateCU
    ,REPLACE(sku,'F','') as CU,REPLACE(sku,'F','')*60*24 as CUDay
    ,SUM(TotalCUs) as SumCUWorkspace
    FROM [dbo].[FUAM_capacity_metrics_by_item_kind_by_day] as WorkspaceCapacityUsage 
    JOIN [dbo].[FUAM_workspaces] on WorkspaceCapacityUsage.WorkspaceId = FUAM_workspaces.WorkspaceId
    JOIN [dbo].[FUAM_capacities] on WorkspaceCapacityUsage.CapacityId = FUAM_capacities.CapacityId
    -- All FABRIC F PAID CAPACITIES
    WHERE FUAM_capacities.sku LIKE('F%') AND FUAM_capacities.sku NOT LIKE('FT%')
    GROUP BY FUAM_capacities.CapacityId, FUAM_capacities.displayName, WorkspaceCapacityUsage.WorkspaceId, FUAM_workspaces.WorkspaceName, WorkspaceCapacityUsage.Date, REPLACE(sku,'F',''),REPLACE(sku,'F','')*60*24
    )as TotalCU
    """
    # Execute 
    sql.query(query)

In [ ]:
# Create View v_FabricCostSplitByWorkspace = Get all Capacity cost by day split by workspace consumption(percentage)

with labs.ConnectLakehouse(lakehouse=destination_lakehouse, workspace=destination_workspace) as sql:
    query = """
    CREATE OR ALTER VIEW [dbo].[v_FabricCostSplitByWorkspace] as 
    SELECT v_CapacityCostPeriod.CapacityId,v_CapacityCostPeriod.CapacityName,WorkspaceId,WorkspaceName,ChargePeriod,TotalCost as TotalCostCapacity,TotalCost*TotalCUWorkspacePercCapacity as TotaCostWorkspace
    FROM [dbo].[v_CapacityCostPeriod]
    LEFT OUTER JOIN [dbo].[v_WorkspacesCUConsumption] on v_CapacityCostPeriod.CapacityName = v_WorkspacesCUConsumption.CapacityName AND DateCU = ChargePeriod
    """
    # Execute 
    sql.query(query)

